## Mouinting Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import Sequential
from keras.layers import Dense,SimpleRNN,Embedding,Flatten
from keras.losses import  CategoricalCrossentropy

## Reading CSV

[Dataset](https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis)

In [16]:
df1 = pd.read_csv('twitter_training.csv', names=['A','B','C','D'])
df2 = pd.read_csv('twitter_validation.csv', names=['A','B','C','D'])
df=pd.concat([df1, df2])
df

,A,B,C,D
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
995,4891,GrandTheftAuto(GTA),Irrelevant,⭐️ Toronto is the arts and culture capital of ...
996,4359,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
997,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
998,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [17]:
words = df.iloc[:,3]
words = np.array(words, 'str')

In [19]:
words.shape

(75682,)

## Splitting the Data

In [20]:
train_X = df1.iloc[:,3]
test_X = df2.iloc[:,3]
train_y = df1.iloc[:,2]
test_y = df2.iloc[:,2]

In [21]:
train_X.head()

0    im getting on borderlands and i will murder yo...
1    I am coming to the borders and I will kill you...
2    im getting on borderlands and i will kill you ...
3    im coming on borderlands and i will murder you...
4    im getting on borderlands 2 and i will murder ...
Name: D, dtype: object

In [22]:
test_y.head()

0    Irrelevant
1       Neutral
2      Negative
3      Negative
4       Neutral
Name: C, dtype: object

In [23]:
train_X = np.array(train_X, 'str')
test_X = np.array(test_X, 'str')

## Count number of words

In [24]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(words)
len(tokenizer.word_index)

34461

In [25]:
train_X = tokenizer.texts_to_sequences(train_X)
test_X = tokenizer.texts_to_sequences(test_X)
test_X[0]

[2,
 2333,
 13,
 78,
 15,
 2,
 29,
 2524,
 9,
 5367,
 3,
 83,
 9,
 5,
 450,
 1,
 244,
 135,
 238,
 42,
 75,
 5443,
 65,
 21826,
 86,
 8672,
 40,
 21827,
 369,
 38,
 31,
 6,
 21828,
 4,
 689,
 3,
 179,
 4635,
 90,
 37,
 1943,
 200,
 5,
 1805,
 533,
 541,
 1806]

## Use encoding technique on Y axis

In [26]:
label_encoder = preprocessing.LabelEncoder()
test_y = label_encoder.fit_transform(test_y)
train_y = label_encoder.fit_transform(train_y)

## Padding to make every sentence with equal length

In [27]:
train_X = pad_sequences(train_X,padding='post',maxlen=50)
test_X = pad_sequences(test_X,padding='post',maxlen=50)

In [28]:
train_X.shape

(74682, 50)

In [29]:
train_X[12]

array([ 1658,   350,  1701, 19074,  3295,   907,  1928,  7912,   139,
          70,    91,   578,    11, 10506,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0], dtype=int32)

In [30]:
test_X[0]

array([    2,  2333,    13,    78,    15,     2,    29,  2524,     9,
        5367,     3,    83,     9,     5,   450,     1,   244,   135,
         238,    42,    75,  5443,    65, 21826,    86,  8672,    40,
       21827,   369,    38,    31,     6, 21828,     4,   689,     3,
         179,  4635,    90,    37,  1943,   200,     5,  1805,   533,
         541,  1806,     0,     0,     0], dtype=int32)

In [32]:
model = Sequential()
model.add(Embedding(33784, 2))
model.add(SimpleRNN(32,return_sequences=False))
model.add(Dense(1, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 2)           67568     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                1120      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 68721 (268.44 KB)
Trainable params: 68721 (268.44 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(train_X, train_y, epochs=5, validation_data=(test_X,test_y))

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


2334/2334 [==============================] - 158s 67ms/step - loss: 0.0000e+00 - acc: 0.3018 - val_loss: 0.0000e+00 - val_acc: 0.2660
Epoch 2/5
 905/2334 [==========>...................] - ETA: 1:12 - loss: 0.0000e+00 - acc: 0.3043